<a href="https://colab.research.google.com/github/VictorVazquezRey/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Victor_Vazquez_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:  Víctor Vázquez Rey

Colab:[https://colab.research.google.com/drive/1dks6zPHfPFqSEay7kkD2qmEyXDf36ydm?usp=sharing](https://colab.research.google.com/drive/1dks6zPHfPFqSEay7kkD2qmEyXDf36ydm?usp=sharing) 

Github: [https://github.com/VictorVazquezRey/03MAIR--Algoritmos-de-Optimizacion--2020](https://github.com/VictorVazquezRey/03MAIR--Algoritmos-de-Optimizacion--2020)

**Problema:**

>**3. Combinar cifras y operaciones**

**Descripción del problema:**

- El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad dada. Un ejemplo sería para obtener el 4:

  - **4+2-6/3*1 = 4**

                                        

**¿Cuantas posibilidades hay sin tener en cuenta las restricciones?**

Respuesta

El número de posibilidades se calcula a partir de la observación de que las posibles colocaciones de números son una variación de los 9 dígitos tomados de 5 en cinco y la de operaciones la de 4 elementos tomados de 4 en cuatro.

Y puesto que el número de variaciones de n elementos tomados de k en k (con $n<k$) es:

$$V_k^n=\frac{n!}{(n-k)!}$$

El número de posibilidades pedido es:

$$Posibilidades=V_5^9\cdot V_4^4=\frac{9!}{5!}\cdot\frac{4!}{0!}=362.880$$

**Modelo para el espacio de soluciones**

(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Una estructura de datos que podría modelar correctamente el estado de la solución en cualquier momento de la búsqueda es una tupla con tres elementos. Dos listas que representan los números y las operaciones respectivamente y una ristra con la expresión(inicialmente había empleado un entero que representa el valor calculado de las operaciones, pero puesto que en python puedo usar eval el uso del entero se había quedado inoperativo debido a que se hacía siempre una única llamada a eval).

Así la tupla $([2,3], ['*'], '2*3')$ representa a el estado $2*3=6$



**Según el modelo para el espacio de soluciones**
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Si definimos la función objetivo $f(x)$ como la distancia del valor objetivo n al valor calculado k entonces $f(x)=|n-k|$ y **el problema es de minimización** puesto que se pretende encontrar un cero de esta función objetivo.

Para una tupla $x=([2,3], ['*'], '2*3')$ y un valor objetivo de 4, $f(x)=|6-4|=2$

Si llamamos $l_n$ a la lista de números y $l_o$ a la de operaciones en este problema las restricciones son:

- $len(l_n)=k$ con $0\leq k<5$
- $len(l_o)=\begin{cases}
0,  & \text{si $k=0$} \\
k-1, & \text{si $k>0$}
\end{cases}$
- $\forall x\in l_n, x\in\{1,2,3,4,5,6,7,8,9\}$
- $\forall y\in l_o, y\in\{+,-,*,/\}$

**Diseña un algoritmo para resolver el problema por fuerza bruta**

Voy a utilizar la clase Node como auxiliar. Esta clase va a contener la información relativa a un nodo, que incluye dos listas (de números y operaciones) y la expresión como una rista.

E incluye diferentes métodos. Los métodos que van a utilizar los clientes de esta clase (puesto que el resto son de operación interna) son el constructor, los métodos constructivos add_number y add_operation y eval_node que devuelve el valor de la expresión que representa el nodo. 

In [2]:
class Node:
    def __init__(self, numbers:list=[], operations:list=[]):
        self.numbers = numbers
        self.operations = operations
        self.expression = self.generate_expression()

    def eval_node(self):
        if not self.numbers:
            return None
        return eval(self.expression)

    @staticmethod
    def is_empty(l: list):
        return not l

    def generate_expression(self):
        if Node.is_empty(self.numbers):
            return ''
        elif Node.is_empty(self.operations):
            return str(self.numbers[0])

        result = str(self.numbers[0])
        for n,o in zip(self.numbers[1:],self.operations):
            result += "{}{}".format(o,n)

        return result

    def add_number(self, number:int):
        self.numbers.append(number)
        self.expression += str(number)

    def add_operation(self,operation:str):
        self.operations.append(operation)
        self.expression += operation
        
    def substact_number(self):
        self.numbers.pop()
        self.expression = self.expression[:-1]

    def substact_operation(self):
        self.operations.pop()
        self.expression = self.expression[:-1]

Esta es la resolución del algoritmo por fuerza bruta que hace uso de la función permutations del módulo itertools que devuelve todas las permutaciones de una lista.

In [3]:
from itertools import permutations

def solve_expresion(value:int)->str:
    OPERATIONS = ['+', '-', '*', '/']
    for n in permutations([i+1 for i in range(9)], len(OPERATIONS) + 1):
        for e in permutations(OPERATIONS):
            node = Node(n,e)
            if node.eval_node()==value:
                return node.expression
        
        
print(solve_expresion(72))

2-6/3+8*9


**Calcula la complejidad del algoritmo por fuerza bruta**

Para calcular la complejidad del algoritmo que resuelve este problema vamos a generalizarlo (puesto que como este tiene longitudes fijas el número de operaciones no es dependiente de un tamaño y por tanto nos daría una complejidad unitaria con una constante multiplicativa exacta) esto quiere decir que no depende del valor que se quiera calcular, el número de operaciones es el mismo por serlo el número de dígitos y de operaciones.

Llamemos $n$ al número de operaciones que vamos a usar en este problema. Siendo así el número de dígitos tiene que ser exactamente $n+1$, y puesto que la operación crítica es la evaluación del bucle más interno se ejecuta $\frac{9!}{(9-n)!}\cdot n!$ veces. Con las restricciónes de no repetición de dígitos y operaciones podemos decir que el coste más alto del algoritmo que resuelve el problema por fuerza bruta está cuando el número de operaciones está en torno a la mitad de los posibles dígitos, y en ese caso tiene un coste que $\in O((n!)^2)$

**¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?**

El valor máximo que puede alcanzar con estas condiciones y teniendo en cuenta la precedencia de operaciones es aquel que maximice la multiplicación y de los que quedan que obtenga la suma máxima.

Así, la multiplicación no puede ser mayor de $72$ que es la multiplicación de $9\cdot8$ adicionalmente es obligatorio usar el $1$ para dividir para maximizar el valor y por último de los números que nos quedan hallar la resta mínima $7-2$. 

Con todo esto el valor máximo es $9\cdot8+7-2 = 77$ que se puede obtener de cualquier permutación de esta misma operación usando el $/1$ en cualquiera de los operandos anteriores por ejemplo: $9/1\cdot8+7-2 = 77$ o $9\cdot8/1+7-2 = 77$.

El valor mínimo el que máximiza la resta y minimiza el resto de operaciones y por tanto el restando debe ser "$-9\cdot8$", y por tanto para el resto de la expresión "$x+y/z$" debe ser mínimo.

El menor valor para "y/z" es $2$ puesto que $y\neq z$ y la solución debe ser entera y para que $x$ sea mínimo debe ser $x=1$.

Con todo esto el valor mínimo es $-72+1+2=-69$ y una posible solución es: $1+4/2-9\cdot8 = -69$ otra $1+6/3-9\cdot8 = -69$

**¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo?**

Este algoritmo prueba que se puede resolver el problema para todos los valores entre el máximo y el mínimo.

In [4]:
inf_limit = -69
sup_limit = 77

not_reachable = []
for v in range(inf_limit,sup_limit+1):
    if not solve_expresion(v):
        not_reachable.append(v)
print(not_reachable)

[]


(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [5]:
def solve_backtracking_expresion(value:int)->str:
    # Explorando las posibles soluciones se observa que el producto tiene más relevancia
    # al igual que los números más altos en las soluciones y por eso se exploran primero.
    # (el orden de exploración es inverso al de la lista por el uso de pop y append).
    OPERATIONS = ['-','/','*','+']
    NUMBERS = [i for i in range(9,0,-1)]
    inf_limit = -69
    sup_limit = 77

    def solve(value:int, node:Node):
        def solve_for_number(value:int, node:Node):
            for n in set(NUMBERS)-set(node.numbers):
                node.add_number(n)
                if solve(value, node):
                    return True
                node.substact_number()
            return False

        if len(node.operations) == 4:
            if node.eval_node()==value:
                return True
            else:
                return False
        elif not node.numbers:
            return solve_for_number(value,node)
        else:
            for o in set(OPERATIONS)-set(node.operations):
                node.add_operation(o)
                if solve_for_number(value, node):
                    return True
                node.substact_operation()
        return False

    if value < inf_limit or value > sup_limit:
        raise ValueError('Valor imposible de calcular')

    node = Node()
    if solve(value, node):
        return node.expression
    else:
        return ""
    
print(solve_backtracking_expresion(-60))


1-7*9+4/2


(*)Calcula la complejidad del algoritmo 

En el peor caso la resolución por backtracking tiene una complejidad $\in O(n!)$ igual que la resolución por fuerza bruta, pero en el caso promedio será de $\in O(2^n)$

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

En este algoritmo no tiene sentido hablar de unos datos de entrada aleatorios.

Aplica el algoritmo al juego de datos generado

No tiene sentido.

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

No he usado más referencias que los apuntes de la asignatura.

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Es posible aplicar heurísticas que poden en caso de que el número no sea entero con la terna $x\cdot y/z$. En estos casos z no puede ser un primo mayor que 1.

Si se pueden repetir números y operaciones el problema crece a $\in O(9^n\cdot 4^{n-1})$ siendo n el número de operandos.